In [ ]:
!pip install google-generativeai

import os
import json
import google.generativeai as genai

class InteractiveResearchClarifier:
    def __init__(self, api_key, max_tokens=8000):
        self.max_tokens = max_tokens
        self.current_tokens = 0
        self.max_iterations = 5 #5 is a limit set by me for reserving tokens for more research , can be changed if wanted
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-flash') #less tokens will be used , please use gemini 1.5 models.

    def estimate_tokens(self, text):
        return len(text.split()) * 1.3

    def check_token_balance(self, additional_tokens):
        return (self.current_tokens + additional_tokens) < self.max_tokens

    def analyze_query_sufficiency(self, conversation_context):
        prompt_tokens = self.estimate_tokens(conversation_context)

        if not self.check_token_balance(prompt_tokens):
            return {"status": "insufficient_tokens"}

        analysis_prompt = f"""
Analyze this research conversation: "{conversation_context}"

Determine if there is now enough information to start research or if more clarification is needed.

Consider the original query and all follow-up answers provided.

Respond in exact JSON format:
{{
    "sufficient_for_research": true or false,
    "research_topic": "brief summary of what will be researched",
    "follow_up_questions": ["specific question 1", "specific question 2"]
}}
"""

        try:
            response = self.model.generate_content(analysis_prompt)
            self.current_tokens += prompt_tokens

            response_text = response.text.strip()

            if "json" in response_text:
                lines = response_text.split('\n')
                json_lines = []
                capture = False
                for line in lines:
                    if line.strip().startswith('{'):
                        capture = True
                    if capture:
                        json_lines.append(line)
                    if line.strip().endswith('}'):
                        break
                response_text = '\n'.join(json_lines)

            result = json.loads(response_text)

            if "sufficient_for_research" not in result:
                result["sufficient_for_research"] = False
                result["follow_up_questions"] = ["Could you provide more details?"]

            return result

        except Exception:
            return {
                "sufficient_for_research": False,
                "research_topic": "",
                "follow_up_questions": ["Could you provide more specific details?"]
            }

    def interactive_research_clarification(self):
        if not self.check_token_balance(100):
            print("Insufficient token balance to process requests.")
            return

        print("What do you want to research?")
        initial_query = input().strip()

        if not initial_query:
            print("Please provide a valid research query.")
            return

        conversation_context = f"Original query: {initial_query}"
        iteration = 0

        while iteration < self.max_iterations:
            print(f"\n- Analysis Round {iteration + 1} -")

            analysis = self.analyze_query_sufficiency(conversation_context)

            if "status" in analysis and analysis["status"] == "insufficient_tokens":
                print("Insufficient token balance for this query.")
                break

            if analysis["sufficient_for_research"]:
                print("\n SUCCESS: Your query is now sufficient for research!")
                print(f" Research Topic: {analysis['research_topic']}")
                print("\n Let's start the research phase!")
                break
            else:
                print("\n More information needed:")
                questions = analysis.get("follow_up_questions", [])

                if not questions:
                    print("No specific questions generated. Please provide more details about your research needs.")
                    additional_info = input("Additional information: ").strip()
                    if additional_info:
                        conversation_context += f" Additional info: {additional_info}"
                else:
                    for i, question in enumerate(questions, 1):
                        print(f"{i}. {question}")

                    print(f"\nPlease answer question 1:")
                    answer = input("Your answer: ").strip()

                    if answer:
                        conversation_context += f" Answer to '{questions[0]}': {answer}"
                    else:
                        print("No answer provided. Ending clarification process.")
                        break

            iteration += 1

        if iteration >= self.max_iterations:
            print(f"\n Maximum clarification rounds ({self.max_iterations}) reached.")
            print("Proceeding with available information:")
            print(f" Research Context: {conversation_context}")
            print("\n Starting research phase with current information!")

    def quick_test(self):
        test_queries = [
            "What are the best ways to travel from Mumbai to Goa?",
            "Research about AI",
            "Is Ayush better than AI?",
            "Compare programming languages"
        ]

        print(" Quick Test Mode - Testing different query types:\n")

        for i, query in enumerate(test_queries, 1):
            print(f"Test {i}: {query}")
            analysis = self.analyze_query_sufficiency(f"Original query: {query}")

            if analysis["sufficient_for_research"]:
                print(f" Sufficient - Topic: {analysis['research_topic']}")
            else:
                print(f" Needs clarification:")
                for j, q in enumerate(analysis["follow_up_questions"], 1):
                    print(f"   {j}. {q}")
            print()

os.environ['GEMINI_API_KEY'] = 'Insert your gemini api key here'

clarifier = InteractiveResearchClarifier(os.environ['GEMINI_API_KEY'])

print(" Interactive deepresearcher firststage:")
print("Choose mode:")
print("1. Interactive Mode (Full clarification process)")
print("2. Quick Test Mode (Test different query types)")

choice = input("Enter 1 or 2: ").strip()

if choice == "1":
    clarifier.interactive_research_clarification()
elif choice == "2":
    clarifier.quick_test()
else:
    print("Invalid choice. Starting interactive mode by default.")
    clarifier.interactive_research_clarification()

🔬 INTERACTIVE RESEARCH CLARIFIER
Choose mode:
1. Interactive Mode (Full clarification process)
2. Quick Test Mode (Test different query types)
Enter 1 or 2: 2
🧪 Quick Test Mode - Testing different query types:

Test 1: What are the best ways to travel from Mumbai to Goa?
❓ Needs clarification:
   1. What is the traveler's budget?
   2. What is the traveler's preferred mode of transportation (e.g., train, bus, car, flight)?
   3. How much time does the traveler have for the journey?
   4. How many people are traveling?
   5. What is the traveler's definition of 'best' (e.g., fastest, cheapest, most comfortable)?

Test 2: Research about AI
❓ Needs clarification:
   1. What specific aspect of AI are you interested in? (e.g., AI ethics, AI in healthcare, AI algorithms, the history of AI, etc.)
   2. What is the scope of your research? (e.g., a literature review, a comparative analysis, a case study, etc.)

Test 3: Is Rahul better than Vishal?
❓ Needs clarification:
   1. Better in what asp